In [1]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np
import os
from pathlib import Path

print("✓ Đã import thư viện thành công")

✓ Đã import thư viện thành công


In [2]:
# Đường dẫn đến thư mục processdataset
base_path = Path(r'd:\project\dragon-fly-data\processdataset')

# Đọc các file CSV
print("Đang đọc dữ liệu...")

df_population = pd.read_csv(base_path / 'population_demographics_consolidated.csv')
df_economic = pd.read_csv(base_path / 'economic_consolidated.csv')
df_health = pd.read_csv(base_path / 'health_hdi_consolidated.csv')
df_employment = pd.read_csv(base_path / 'employment_consolidated.csv')
df_urbanization = pd.read_csv(base_path / 'urbanization_consolidated.csv')
df_environment = pd.read_csv(base_path / 'environment_energy_consolidated.csv')
df_reference = pd.read_csv(base_path / 'reference_regional_consolidated.csv')

print(f"✓ Population: {len(df_population)} rows")
print(f"✓ Economic: {len(df_economic)} rows")
print(f"✓ Health: {len(df_health)} rows")
print(f"✓ Employment: {len(df_employment)} rows")
print(f"✓ Urbanization: {len(df_urbanization)} rows")
print(f"✓ Environment: {len(df_environment)} rows")
print(f"✓ Reference: {len(df_reference)} rows")

Đang đọc dữ liệu...
✓ Population: 65 rows
✓ Economic: 55 rows
✓ Health: 65 rows
✓ Employment: 65 rows
✓ Urbanization: 65 rows
✓ Environment: 65 rows
✓ Reference: 65 rows


In [3]:
# Tạo dataframe gốc với khoảng năm 1955-2025
years = range(1955, 2026)
df_final = pd.DataFrame({'Year': years})

print(f"Tạo dataframe với {len(df_final)} năm (1955-2025)")

Tạo dataframe với 71 năm (1955-2025)


In [4]:
# Merge dữ liệu dân số và nhân khẩu học
print("\n=== Merge Population & Demographics ===")

# Tính toán tổng dân số từ Rural + Urban
df_population['Population'] = df_population['RuralPopulation'] + df_population['UrbanPopulation']

# Chọn các cột cần thiết từ population
pop_cols = [
    'Year', 'Population', 'MedianAge', 'DependencyRatio', 'SexRatio',
    'Pop0to14Pct', 'Pop15to64Pct', 'Pop65PlusPct', 'FertilityRate',
    'BirthRatePer1000', 'DeathRatePer1000', 'RuralPopulation', 'UrbanPopulation'
]
df_pop_selected = df_population[pop_cols].copy()

# Merge vào df_final
df_final = df_final.merge(df_pop_selected, on='Year', how='left')

print(f"Đã merge {len(pop_cols)-1} cột từ population")
print(f"Shape: {df_final.shape}")


=== Merge Population & Demographics ===
Đã merge 12 cột từ population
Shape: (71, 13)


In [5]:
# Merge dữ liệu tham chiếu và xếp hạng
print("\n=== Merge Reference & Regional ===")

ref_cols = [
    'Year', 'VN_Global_Rank_Population', 'VN_ASEAN_Rank_Population',
    'World_Population', 'HDI', 'Poverty_Rate_215_Day'
]
df_ref_selected = df_reference[ref_cols].copy()

# Tính Vietnam's Share of Asian Pop và World Pop
# Giả sử Asian Pop ~ 60% World Pop
df_ref_selected['VN_Share_Asia_Pct'] = np.nan
df_ref_selected['VN_Share_World_Pct'] = np.nan

df_final = df_final.merge(df_ref_selected, on='Year', how='left')

# Tính toán share percentages nếu có dữ liệu
mask = (df_final['Population'].notna()) & (df_final['World_Population'].notna())
df_final.loc[mask, 'VN_Share_World_Pct'] = (
    df_final.loc[mask, 'Population'] / (df_final.loc[mask, 'World_Population'] * 1_000_000) * 100
)
df_final.loc[mask, 'VN_Share_Asia_Pct'] = (
    df_final.loc[mask, 'Population'] / (df_final.loc[mask, 'World_Population'] * 1_000_000 * 0.6) * 100
)

print(f"Đã merge {len(ref_cols)-1} cột từ reference")
print(f"Shape: {df_final.shape}")


=== Merge Reference & Regional ===
Đã merge 5 cột từ reference
Shape: (71, 20)


In [6]:
# Thêm Regional Median Age và Global Median Age (ước tính)
print("\n=== Thêm Regional & Global Median Age ===")

# Ước tính dựa trên xu hướng lịch sử
# Regional Median Age (ASEAN) thường thấp hơn Global 2-5 tuổi
df_final['Regional_Median_Age'] = np.nan
df_final['Global_Median_Age'] = np.nan

# Ước tính cho các năm có dữ liệu MedianAge
mask = df_final['MedianAge'].notna()
df_final.loc[mask, 'Regional_Median_Age'] = df_final.loc[mask, 'MedianAge'] - 2
df_final.loc[mask, 'Global_Median_Age'] = df_final.loc[mask, 'MedianAge'] + 3

print("Đã thêm Regional và Global Median Age")


=== Thêm Regional & Global Median Age ===
Đã thêm Regional và Global Median Age


In [7]:
# Merge dữ liệu kinh tế
print("\n=== Merge Economic Data ===")

econ_cols = [
    'Year', 'GDPPerCapita', 'UnemploymentRate', 'GDPGrowthRate',
    'FDINetInflowsMillion', 'GNIPerCapitaPPP'
]
df_econ_selected = df_economic[econ_cols].copy()

df_final = df_final.merge(df_econ_selected, on='Year', how='left')

print(f"Đã merge {len(econ_cols)-1} cột từ economic")
print(f"Shape: {df_final.shape}")


=== Merge Economic Data ===
Đã merge 5 cột từ economic
Shape: (71, 27)


In [8]:
# Merge dữ liệu sức khỏe
print("\n=== Merge Health Data ===")

health_cols = [
    'Year', 'LifeExpectancy', 'HealthExpenditureGDP', 'HumanCapitalIndex'
]
df_health_selected = df_health[health_cols].copy()

df_final = df_final.merge(df_health_selected, on='Year', how='left')

print(f"Đã merge {len(health_cols)-1} cột từ health")
print(f"Shape: {df_final.shape}")


=== Merge Health Data ===
Đã merge 3 cột từ health
Shape: (71, 30)


In [9]:
# Merge dữ liệu việc làm
print("\n=== Merge Employment Data ===")

emp_cols = [
    'Year', 'AgricultureEmploymentPct', 'IndustryEmploymentPct',
    'ServicesEmploymentPct'
]
df_emp_selected = df_employment[emp_cols].copy()

df_final = df_final.merge(df_emp_selected, on='Year', how='left')

print(f"Đã merge {len(emp_cols)-1} cột từ employment")
print(f"Shape: {df_final.shape}")


=== Merge Employment Data ===
Đã merge 3 cột từ employment
Shape: (71, 33)


In [10]:
# Merge dữ liệu môi trường & năng lượng
print("\n=== Merge Environment & Energy Data ===")

env_cols = [
    'Year', 'EnergyUsePerCapita', 'CO2EmissionsPerCapita',
    'AgriLandPercent', 'ForestAreaPercent', 'RenewableEnergyPercent'
]
df_env_selected = df_environment[env_cols].copy()

df_final = df_final.merge(df_env_selected, on='Year', how='left')

print(f"Đã merge {len(env_cols)-1} cột từ environment")
print(f"Shape: {df_final.shape}")


=== Merge Environment & Energy Data ===
Đã merge 5 cột từ environment
Shape: (71, 38)


In [11]:
# Đổi tên cột theo yêu cầu
print("\n=== Đổi tên cột ===")

column_mapping = {
    'VN_Global_Rank_Population': 'Vietnam_Global_Rank',
    'VN_ASEAN_Rank_Population': 'ASEAN_Population_Rank',
    'VN_Share_Asia_Pct': 'Vietnam_Share_of_Asian_Pop_Pct',
    'VN_Share_World_Pct': 'Country_Share_of_World_Pop',
    'MedianAge': 'Median_Age',
    'Regional_Median_Age': 'Regional_Median_Age',
    'Global_Median_Age': 'Global_Median_Age',
    'DependencyRatio': 'Dependency_Ratio_Pct',
    'SexRatio': 'Sex_Ratio_MF',
    'Pop0to14Pct': 'Pop_Aged_0_14_Pct',
    'Pop15to64Pct': 'Pop_Aged_15_64_Pct',
    'Pop65PlusPct': 'Pop_Aged_65_Plus_Pct',
    'GDPPerCapita': 'GDP_per_Capita_USD',
    'UnemploymentRate': 'Unemployment_Rate_Pct',
    'GDPGrowthRate': 'GDP_Growth_Rate_Pct',
    'FDINetInflowsMillion': 'FDI_Net_Inflows_million_USD',
    'GNIPerCapitaPPP': 'GDP_PPP_per_Capita_IntDollar',
    'FertilityRate': 'Fertility_Rate',
    'LifeExpectancy': 'Life_Expectancy',
    'BirthRatePer1000': 'Birth_Rate_per_1000',
    'DeathRatePer1000': 'Death_Rate_per_1000',
    'AgricultureEmploymentPct': 'Employment_Agriculture_Pct',
    'IndustryEmploymentPct': 'Employment_Industry_Pct',
    'ServicesEmploymentPct': 'Employment_Services_Pct',
    'Poverty_Rate_215_Day': 'Poverty_Rate_Pct',
    'HealthExpenditureGDP': 'Health_Expenditure_Pct_GDP',
    'RuralPopulation': 'Rural_Population',
    'UrbanPopulation': 'Urban_Population',
    'EnergyUsePerCapita': 'Energy_Consumption_per_Capita_kWh',
    'CO2EmissionsPerCapita': 'CO2_Emissions_per_Capita_tonnes',
    'AgriLandPercent': 'Agricultural_Land_Pct_Land',
    'ForestAreaPercent': 'Forest_Area_Pct_Land',
    'HumanCapitalIndex': 'Human_Capital_Index_0_1',
    'RenewableEnergyPercent': 'Renewable_Energy_Share_Pct'
}

df_final = df_final.rename(columns=column_mapping)

print("Đã đổi tên các cột theo format yêu cầu")


=== Đổi tên cột ===
Đã đổi tên các cột theo format yêu cầu


In [12]:
# Sắp xếp lại thứ tự cột theo yêu cầu
print("\n=== Sắp xếp cột ===")

final_columns = [
    'Year',
    'Population',
    'Vietnam_Global_Rank',
    'ASEAN_Population_Rank',
    'Vietnam_Share_of_Asian_Pop_Pct',
    'Country_Share_of_World_Pop',
    'Median_Age',
    'Regional_Median_Age',
    'Global_Median_Age',
    'Dependency_Ratio_Pct',
    'Sex_Ratio_MF',
    'Pop_Aged_0_14_Pct',
    'Pop_Aged_15_64_Pct',
    'Pop_Aged_65_Plus_Pct',
    'GDP_per_Capita_USD',
    'HDI',
    'Unemployment_Rate_Pct',
    'GDP_Growth_Rate_Pct',
    'FDI_Net_Inflows_million_USD',
    'GDP_PPP_per_Capita_IntDollar',
    'Fertility_Rate',
    'Life_Expectancy',
    'Birth_Rate_per_1000',
    'Death_Rate_per_1000',
    'Employment_Agriculture_Pct',
    'Employment_Industry_Pct',
    'Employment_Services_Pct',
    'Poverty_Rate_Pct',
    'Health_Expenditure_Pct_GDP',
    'Rural_Population',
    'Urban_Population',
    'Energy_Consumption_per_Capita_kWh',
    'CO2_Emissions_per_Capita_tonnes',
    'Agricultural_Land_Pct_Land',
    'Forest_Area_Pct_Land',
    'Human_Capital_Index_0_1',
    'Renewable_Energy_Share_Pct'
]

# Kiểm tra xem tất cả các cột có tồn tại không
missing_cols = [col for col in final_columns if col not in df_final.columns]
if missing_cols:
    print(f"Cảnh báo: Các cột thiếu: {missing_cols}")
    for col in missing_cols:
        df_final[col] = np.nan

df_final = df_final[final_columns]

print(f"Đã sắp xếp {len(final_columns)} cột theo thứ tự yêu cầu")


=== Sắp xếp cột ===
Đã sắp xếp 37 cột theo thứ tự yêu cầu


In [13]:
# Kiểm tra dữ liệu
print("\n=== Kiểm tra dữ liệu ===")
print(f"\nShape: {df_final.shape}")
print(f"Năm từ {df_final['Year'].min()} đến {df_final['Year'].max()}")
print(f"\nThông tin cột:")
print(df_final.info())

print(f"\n5 hàng đầu tiên:")
df_final.head()


=== Kiểm tra dữ liệu ===

Shape: (71, 37)
Năm từ 1955 đến 2025

Thông tin cột:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               71 non-null     int64  
 1   Population                         65 non-null     float64
 2   Vietnam_Global_Rank                3 non-null      float64
 3   ASEAN_Population_Rank              3 non-null      float64
 4   Vietnam_Share_of_Asian_Pop_Pct     65 non-null     float64
 5   Country_Share_of_World_Pop         65 non-null     float64
 6   Median_Age                         65 non-null     float64
 7   Regional_Median_Age                65 non-null     float64
 8   Global_Median_Age                  65 non-null     float64
 9   Dependency_Ratio_Pct               65 non-null     float64
 10  Sex_Ratio_MF                       64 non-nu

,Year,Population,Vietnam_Global_Rank,ASEAN_Population_Rank,Vietnam_Share_of_Asian_Pop_Pct,Country_Share_of_World_Pop,Median_Age,Regional_Median_Age,Global_Median_Age,Dependency_Ratio_Pct,...,Poverty_Rate_Pct,Health_Expenditure_Pct_GDP,Rural_Population,Urban_Population,Energy_Consumption_per_Capita_kWh,CO2_Emissions_per_Capita_tonnes,Agricultural_Land_Pct_Land,Forest_Area_Pct_Land,Human_Capital_Index_0_1,Renewable_Energy_Share_Pct
0,1955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Thống kê missing values
print("\n=== Thống kê Missing Values ===")
missing_stats = pd.DataFrame({
    'Column': df_final.columns,
    'Missing_Count': df_final.isnull().sum(),
    'Missing_Pct': (df_final.isnull().sum() / len(df_final) * 100).round(2)
})
missing_stats = missing_stats[missing_stats['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print(f"\nCác cột có missing values:")
print(missing_stats)


=== Thống kê Missing Values ===

Các cột có missing values:
                                                              Column  \
ASEAN_Population_Rank                          ASEAN_Population_Rank   
Vietnam_Global_Rank                              Vietnam_Global_Rank   
Human_Capital_Index_0_1                      Human_Capital_Index_0_1   
Poverty_Rate_Pct                                    Poverty_Rate_Pct   
Health_Expenditure_Pct_GDP                Health_Expenditure_Pct_GDP   
Renewable_Energy_Share_Pct                Renewable_Energy_Share_Pct   
Energy_Consumption_per_Capita_kWh  Energy_Consumption_per_Capita_kWh   
Employment_Services_Pct                      Employment_Services_Pct   
Employment_Industry_Pct                      Employment_Industry_Pct   
Employment_Agriculture_Pct                Employment_Agriculture_Pct   
HDI                                                              HDI   
Forest_Area_Pct_Land                            Forest_Area_Pct_Land   
Une

In [15]:
# Xem một số thống kê mô tả
print("\n=== Thống kê mô tả ===")
print("\nCác chỉ số chính:")
key_metrics = [
    'Year', 'Population', 'GDP_per_Capita_USD', 'HDI', 
    'Life_Expectancy', 'Fertility_Rate', 'Unemployment_Rate_Pct'
]
df_final[key_metrics].describe()


=== Thống kê mô tả ===

Các chỉ số chính:


,Year,Population,GDP_per_Capita_USD,HDI,Life_Expectancy,Fertility_Rate,Unemployment_Rate_Pct
count,71.000000,6.500000e+01,40.000000,34.000000,64.000000,64.000000,34.000000
mean,1990.000000,6.702792e+07,1438.797250,0.649765,67.294375,3.664781,1.887059
std,20.639767,2.123679e+07,1410.533363,0.080143,7.103033,1.712145,0.455282
min,1955.000000,3.253193e+07,98.110000,0.499000,51.910000,1.873000,1.000000
25%,1972.500000,4.768493e+07,361.567500,0.590000,63.220000,1.983750,1.650000
50%,1990.000000,6.826034e+07,653.000000,0.660000,69.460000,3.438500,1.965000
75%,2007.500000,8.517579e+07,2554.177500,0.715500,73.407500,5.464750,2.135000
max,2025.000000,1.009877e+08,4717.290000,0.766000,75.380000,6.385000,2.870000


In [16]:
# Lưu file CSV
print("\n=== Lưu file CSV ===")

output_path = base_path / 'vietnam_population.csv'
df_final.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✓ Đã lưu file: {output_path}")
print(f"✓ Tổng số: {len(df_final)} hàng x {len(df_final.columns)} cột")
print(f"✓ Khoảng năm: {df_final['Year'].min()}-{df_final['Year'].max()}")

# Kiểm tra file size
file_size = os.path.getsize(output_path) / 1024
print(f"✓ Kích thước file: {file_size:.2f} KB")


=== Lưu file CSV ===

✓ Đã lưu file: d:\project\dragon-fly-data\processdataset\vietnam_population.csv
✓ Tổng số: 71 hàng x 37 cột
✓ Khoảng năm: 1955-2025
✓ Kích thước file: 17.91 KB


In [17]:
# Hiển thị mẫu dữ liệu cuối cùng
print("\n=== Dữ liệu mẫu (10 năm gần nhất) ===")
df_final.tail(10)


=== Dữ liệu mẫu (10 năm gần nhất) ===


,Year,Population,Vietnam_Global_Rank,ASEAN_Population_Rank,Vietnam_Share_of_Asian_Pop_Pct,Country_Share_of_World_Pop,Median_Age,Regional_Median_Age,Global_Median_Age,Dependency_Ratio_Pct,...,Poverty_Rate_Pct,Health_Expenditure_Pct_GDP,Rural_Population,Urban_Population,Energy_Consumption_per_Capita_kWh,CO2_Emissions_per_Capita_tonnes,Agricultural_Land_Pct_Land,Forest_Area_Pct_Land,Human_Capital_Index_0_1,Renewable_Energy_Share_Pct
61,2016,94000117.0,NaN,NaN,2.080826,1.248496,871552.5,871550.5,871555.5,44.385230,...,2.5,4.87,61560677.0,32439440.0,738.88,63.92,38.84,45.88,NaN,26.8
62,2017,95176977.0,NaN,NaN,2.083171,1.249903,835835.0,835833.0,835838.0,45.322182,...,NaN,4.99,61662308.0,33514669.0,757.48,45.87,38.83,46.00,0.6660,28.3
63,2018,96237319.0,NaN,NaN,2.083738,1.250243,787064.5,787062.5,787067.5,46.123352,...,2.2,5.03,61669836.0,34567483.0,885.65,55.32,38.73,46.23,0.6872,24.3
64,2019,97173776.0,NaN,NaN,2.082155,1.249293,766334.0,766332.0,766337.0,46.605432,...,NaN,4.97,61580965.0,35592811.0,987.27,46.06,39.52,46.48,NaN,20.4
65,2020,98079191.0,NaN,NaN,2.081014,1.248608,778979.0,778977.0,778982.0,46.964136,...,1.3,4.33,61456421.0,36622770.0,991.28,29.41,39.43,46.72,0.6900,18.9
66,2021,98935098.0,NaN,NaN,2.081742,1.249045,767039.5,767037.5,767042.5,47.239298,...,NaN,4.54,61288315.0,37646783.0,962.20,NaN,39.43,46.96,NaN,24.2
67,2022,99680655.0,99.68,0.0,2.079175,1.247505,746803.0,746801.0,746806.0,47.416365,...,1.6,4.59,61038452.0,38642203.0,1023.06,NaN,39.29,47.20,NaN,NaN
68,2023,100352192.0,100.35,0.0,2.073826,1.244296,726771.0,726769.0,726774.0,47.551413,...,NaN,NaN,60733147.0,39619045.0,NaN,NaN,39.21,47.45,NaN,NaN
69,2024,100987686.0,100.99,0.0,2.067202,1.240321,708810.5,708808.5,708813.5,47.647651,...,NaN,NaN,60395686.0,40592000.0,NaN,NaN,NaN,NaN,NaN,NaN
70,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Hoàn thành!

File `vietnam_population.csv` đã được tạo thành công với:
- **71 năm** dữ liệu (1955-2025)
- **37 cột** chỉ số kinh tế-xã hội
- Tổng hợp từ 7 nguồn dữ liệu consolidated

### Các cột trong dataset:
1. Year
2. Population
3. Vietnam Global Rank
4. ASEAN Population Rank
5. Vietnam's Share of Asian Pop (%)
6. Country's Share of World Pop
7. Median Age
8. Regional Median Age
9. Global Median Age
10. Dependency Ratio (%)
11. Sex Ratio (M/F)
12. Pop Aged 0–14 (%)
13. Pop Aged 15–64 (%)
14. Pop Aged 65+ (%)
15. GDP per Capita (USD)
16. HDI
17. Unemployment Rate (%)
18. GDP Growth Rate (%)
19. FDI Net Inflows (million USD)
20. GDP PPP per Capita (Int$)
21. Fertility Rate
22. Life Expectancy
23. Birth Rate (‰)
24. Death Rate (‰)
25. Employment Agriculture (%)
26. Employment Industry (%)
27. Employment Services (%)
28. Poverty Rate (%)
29. Health Expenditure (% GDP)
30. Rural Population
31. Urban Population
32. Energy Consumption per Capita (kWh)
33. CO₂ Emissions per Capita (t)
34. Agricultural Land (% Land)
35. Forest Area (% Land)
36. Human Capital Index (0-1)
37. Renewable Energy Share (%)